In [ ]:
ls

In [ ]:
%cd /content/drive/My Drive/task/

/content/drive/My Drive/task


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error, r2_score, roc_auc_score, roc_curve, classification_report, log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LassoCV
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from vecstack import stacking

In [ ]:
!pip install vecstack

In [ ]:
!pip install lightgbm

In [ ]:
!pip install catboost

In [ ]:
#creating a DL model for multiclass classification 
def build_keras_model_1():
    model = Sequential()
    model.add(Dense(256, 
                    input_dim=X_train.shape[1], 
                    kernel_initializer='normal', 
                    activation='relu'))
    model.add(Dense(64, 
                    kernel_initializer='normal', 
                    activation='relu'))
    model.add(Dense(64, 
                    kernel_initializer='normal', 
                    activation='relu'))
    model.add(Dense(64, 
                    kernel_initializer='normal', 
                    activation='relu'))
    model.add(Dense(9, 
                    kernel_initializer='normal', 
                    activation='softmax'))
    model.compile(optimizer='rmsprop', 
                  loss='categorical_crossentropy', 
                  metrics=['categorical_accuracy'])
    return model

In [ ]:
# Create list: stacked_models
stacked_models = [LogisticRegression(random_state=42),
                  KNeighborsClassifier(),
                  CatBoostClassifier(random_state=42),
                  BaggingClassifier(random_state=42,n_jobs=-1),
                  MLPClassifier(random_state=42),
                  AdaBoostClassifier(random_state=42),
                  SVC(probability=True, random_state=42),
                  DecisionTreeClassifier(random_state=42),
                  KerasClassifier(build_fn=build_keras_model_1, epochs=20, batch_size=32, verbose=2),
                  LGBMClassifier(random_state=42,n_jobs=-1)]

#reading training dataset
df = pd.read_csv("train.csv")

#statistic information of dataset
df.head()
df.describe().T

#checking missing data
df.isnull().sum()

#spliting dependent and independent variables
y = df["target"]
X = df.drop(["target"], axis= 1)

#spliting test and train datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

#scaling the dataset
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

#reading validation dataset
df_test = pd.read_csv("validation.csv")

# Stack the models: stack_train, stack_test
stack_train, stack_test = stacking(stacked_models, X_train, y_train, X_test, 
                                   regression=False, mode= 'oof_pred_bag',
                                   needs_proba=True, metric=log_loss, 
                                   n_folds=5, stratified=True, shuffle=True, 
                                   random_state=42, verbose=2)

# Initialize and fit 2nd level model
#final_model = XGBClassifier(n_estimators= 500, max_depth=30, learning_rate=0.01, random_state=42, n_jobs=-1) #0.52
final_model = XGBClassifier(random_state=42, n_jobs=-1)
final_model_fit = final_model.fit(stack_train, y_train)
# Predict: stacked_pred

y_pred = final_model.predict_proba(stack_test)
#y_pred_test = final_model.predict_proba(df_test)

#measuring the error
log_loss(y_test, y_pred)
print(log_loss(y_test, y_pred))